파이썬의 requests 모듈로 .get으로 서버 호출 가능  
text 메서드로 html 내용 조회가능  
(200은 이상없음, 404 이상있다는 뜻)   
beautifulSoup을 쓰는 이유는 html 문서 파싱 (정리하기, 대량의 데이터 신속하게 수집)  
하지만 대부분 동적 크롤링이니까 selenium  

In [1]:
!pip install selenium
!pip install webdriver_manager

css 선택자와 웹사이트 구조에 대한 경험치
스크롤 다운  


import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

# Configure Chrome options for headless execution
options = Options()
options.add_argument('--headless=new')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-gpu')
options.add_argument('--incognito')

# Set up ChromeDriver via webdriver_manager (works with Selenium 4.x)
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Example navigation
driver.get('https://www.naver.com')

# Search for "hello world"
search_input = driver.find_element(By.CLASS_NAME, 'search_input')
search_input.click()
search_input.send_keys('hello world')
search_input.send_keys(Keys.ENTER)

# Wait briefly for results
time.sleep(3)
print(driver.title)

driver.quit()


In [2]:
from IPython.display import display
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

url = "https://securities.koreainvestment.com/main/mall/openels/EdlsInfo.jsp?cmd=TF02cc000000_Main"


def create_driver(headless: bool = True) -> webdriver.Chrome:
    options = Options()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver_path = ChromeDriverManager().install()

    try:
        service = Service(driver_path)
        return webdriver.Chrome(service=service, options=options)
    except TypeError as exc:
        message = str(exc)
        if "multiple values for argument 'options'" in message or "unexpected keyword argument 'service'" in message:
            # Fallback for legacy selenium versions that predate the Service API.
            return webdriver.Chrome(executable_path=driver_path, options=options)
        raise


driver = create_driver(headless=True)
table_df = None

try:
    wait = WebDriverWait(driver, 20)
    driver.get(url)
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe#ifrm_list")))
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#list_style tbody tr")))

    table_html = driver.find_element(By.CSS_SELECTOR, "#list_style table").get_attribute("outerHTML")
    for df in pd.read_html(table_html):
        cleaned = df.dropna(how="all").dropna(axis=1, how="all")
        if cleaned.empty or cleaned.shape[1] < 3:
            continue
        cleaned.columns = [str(col).strip() for col in cleaned.columns]
        table_df = cleaned.reset_index(drop=True)
        break
finally:
    try:
        driver.switch_to.default_content()
    except Exception:
        pass
    driver.quit()

if table_df is None:
    raise RuntimeError("Unable to parse ELS table on page 1.")

display(table_df)


C:\Users\Dongjun\AppData\Local\Temp\ipykernel_26856\575510013.py:45: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  for df in pd.read_html(table_html):


,비교 담기,상품명 오름차순정렬 내림차순정렬,기초자산,구조 오름차순정렬 내림차순정렬,조건충족시 수익률(세전) 오름차순정렬 내림차순정렬,조건미충족시 최대손실률 오름차순정렬 내림차순정렬,상환주기,원금,발행일,상환여부
0,TRUE ELB 제2517 회 (Digital Call) 비교담기 선택,원금지급형 TRUE ELB 제2517 회 (Digital Call),삼성전자,"500%이상 : 3.2919%(연3.301%), 500%미만 : 3.2909%(연3...",연 최대 3.301%,0%,364일/,비보장,2025.11.07,미상환
1,TRUE ELB 제2516 회 (Digital Call) 비교담기 선택,원금지급형 TRUE ELB 제2516 회 (Digital Call),삼성전자,"500%이상 : 1.6459%(연3.301%), 500%미만 : 1.6454%(연3...",연 최대 3.301%,0%,182일/,지급,2025.11.07,미상환
2,TRUE ELB 제2510 회 (Hi-Five) 비교담기 선택,원금지급형 TRUE ELB 제2510 회 (Hi-Five),KOSPI200 현대차,102-102-102-102-102-102-102-102-102-102-102-10...,연 6.51%,0%,3년/6-1,지급,2025.11.07,미상환
3,TRUE ELS 제18271 회 (스텝다운) 비교담기 선택,원금비보장종목형 TRUE ELS 제18271 회 (스텝다운),NVIDIA CORPORATION BROADCOM,80-80-75-70-65-50/30(종가),연 10.3%,-100%,3년/6개월,비보장,2025.11.07,미상환
4,TRUE ELS 제18270 회 (스텝다운) 비교담기 선택,원금비보장종목형 TRUE ELS 제18270 회 (스텝다운),PALANTIR TECHNOLOGIES INC. CLASS A ALPHABET-A,80-80-75-70-65-50/25(종가),연 12.5%,-100%,3년/6개월,비보장,2025.11.07,미상환
5,TRUE ELS 제18269 회 (무조건 월지급식 스텝다운) 비교담기 선택,원금비보장종목형 TRUE ELS 제18269 회 (무조건 월지급식 스텝다운),PALANTIR TECHNOLOGIES INC. CLASS A S&P500,85-85-85-80-70-60/30(종가),연 9.51%,-100%,3년/6개월,비보장,2025.11.07,미상환
6,TRUE ELS 제18268 회 (스텝다운) 비교담기 선택,원금비보장종목형 TRUE ELS 제18268 회 (스텝다운),TESLA PALANTIR TECHNOLOGIES INC. CLASS A,70-70-70-70-65-50/25(종가),연 13.1%,-100%,3년/6개월,비보장,2025.11.07,미상환
7,TRUE ELS 제18267 회 (스텝다운) 비교담기 선택,원금비보장종목형 TRUE ELS 제18267 회 (스텝다운),TESLA NVIDIA CORPORATION,75-75-70-70-65-50/25(종가),연 11.2%,-100%,3년/6개월,비보장,2025.11.07,미상환
8,TRUE ELS 제18266 회 (스텝다운) 비교담기 선택,원금비보장종목형 TRUE ELS 제18266 회 (스텝다운),TESLA S&P500,75-75-70-70-65-50/30(종가),연 10.3%,-100%,3년/6개월,비보장,2025.11.07,미상환
9,TRUE ELS 제18265 회 (월지급식 스텝다운) 비교담기 선택,원금비보장종목형 TRUE ELS 제18265 회 (월지급식 스텝다운),현대차 SK하이닉스,90-85-80-75-70-60/40(종가),연 최대 15.81%,-100%,3년/6개월,비보장,2025.11.07,미상환


In [3]:
import re

full_table_df = None
total_pages = 1

driver = create_driver(headless=True)

try:
    wait = WebDriverWait(driver, 20)
    driver.get(url)
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, "iframe#ifrm_list")))
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#list_style table")))

    headers = [el.text.strip() for el in driver.find_elements(By.CSS_SELECTOR, "#list_style thead th")]

    end_button = driver.find_element(By.CSS_SELECTOR, ".pager a.btn.end")
    onclick = end_button.get_attribute("onclick") or ""
    match = re.search(r"goPage\('([0-9]+)'\)", onclick)
    total_pages = int(match.group(1)) if match else 1

    def collect_rows():
        rows = []
        for tr in driver.find_elements(By.CSS_SELECTOR, "#list_style tbody tr"):
            cells = tr.find_elements(By.CSS_SELECTOR, "th, td")
            rows.append([' '.join(cell.text.split()) for cell in cells])
        return rows

    def wait_for_page(page_number):
        wait.until(EC.text_to_be_present_in_element((By.CSS_SELECTOR, ".pager .num a.on"), str(page_number)))

    all_rows = []
    wait_for_page(1)
    all_rows.extend(collect_rows())

    for page in range(2, total_pages + 1):
        driver.execute_script("goPage(arguments[0]);", str(page))
        wait_for_page(page)
        all_rows.extend(collect_rows())

    full_table_df = pd.DataFrame(all_rows, columns=headers)
finally:
    driver.quit()

if full_table_df is None or full_table_df.empty:
    raise RuntimeError("Failed to retrieve full ELS table.")

display(full_table_df.head())
print(f"Loaded {len(full_table_df)} rows across {total_pages} pages.")
 

TimeoutException: Message: 
